# Stacked Generalizations

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.svm as skl_svm
import sklearn.cross_validation as skl_cv
import seaborn as sns
import os
import sys

base_path = '/home/lundi/Python/MNIST/'
sys.path.append(base_path + '/libraries/')

import time
import glob

import MNIST_model_functions as mmf
MNIST_model_functions = mmf.MNIST_model_functions()

import sklearn.linear_model as skl_lm

In [2]:
prediction_data_v1 = pd.DataFrame()

for filename in glob.glob(base_path + '/data/prediction_results/2016.11.7-*_results.csv'):
    prediction_data_v1 = pd.concat([prediction_data_v1, pd.read_csv(filename)])
prediction_data_v1 = prediction_data_v1.rename(columns = {'Unnamed: 0': 'datum_index'})
#prediction_data_v1 = prediction_data_v1.drop(['Unnamed: 0'], axis = 1)

In [5]:
X = pd.pivot_table(prediction_data_v1, index = 'datum_index', values=['0','1','2','3','4','5','6','7','8','9'], columns=['Model'])
X.columns = ['%s%s' % (a, '|%s' % b if b else '') for a, b in X.columns]

In [33]:
y = prediction_data_v1[['datum_index','Actual']]
y = y.drop_duplicates().set_index('datum_index')
y['Actual'] = y['Actual'].astype(int)

Creating a linear model

In [44]:
lr_clf = skl_lm.LogisticRegression()
skl_cv.cross_val_score(lr_clf, X = X, y = y['Actual'], cv = 10).mean()

0.95122848296585027